In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": ',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<tenant_id>/oauth2/token"}


dbutils.fs.mount(
source = "abfss://contrainer@storageacc.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolympic",
extra_configs = configs)

True

In [0]:
%fs

ls "mnt/tokyoolympic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw-data/,raw-data/,0,1722350153000
dbfs:/mnt/tokyoolympic/transformed-data/,transformed-data/,0,1722350170000


In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
from pyspark.sql.functions import col

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TokyoOlympic").getOrCreate()

In [0]:
dbutils.fs.ls("dbfs:/mnt/tokyoolympic/raw-data/")

[FileInfo(path='dbfs:/mnt/tokyoolympic/raw-data/atheletes.csv', name='atheletes.csv', size=418492, modificationTime=1722351845000),
 FileInfo(path='dbfs:/mnt/tokyoolympic/raw-data/coaches.csv', name='coaches.csv', size=16889, modificationTime=1722351858000),
 FileInfo(path='dbfs:/mnt/tokyoolympic/raw-data/entriesGender.csv', name='entriesGender.csv', size=1123, modificationTime=1722351872000),
 FileInfo(path='dbfs:/mnt/tokyoolympic/raw-data/medals.csv', name='medals.csv', size=2414, modificationTime=1722351886000),
 FileInfo(path='dbfs:/mnt/tokyoolympic/raw-data/teams.csv', name='teams.csv', size=35270, modificationTime=1722351900000)]

In [0]:
athelete_df = spark.read.format("csv") \
            .option("header","true") \
            .option("inferSchema","true") \
            .load("dbfs:/mnt/tokyoolympic/raw-data/atheletes.csv")

athelete_df.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
coaches_df = spark.read.format("csv") \
            .option("header","true") \
            .option("inferschema","true") \
            .load("dbfs:/mnt/tokyoolympic/raw-data/coaches.csv")

coaches_df.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    NULL|
|     

In [0]:
entriesGender_schema = StructType([
    StructField("Discipline",StringType(), True),
    StructField("Female", IntegerType(), True),
    StructField("Male", IntegerType(), True),
    StructField("Total", IntegerType(), True)
])

In [0]:
entriesGender_df = spark.read.format("csv") \
                .option("header","true") \
                .schema(entriesGender_schema) \
                .load("dbfs:/mnt/tokyoolympic/raw-data/entriesGender.csv")

entriesGender_df.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
entriesGender_df.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medal_df = spark.read.format("csv") \
            .option("header","true") \
            .option("inferSchema","true") \
            .load("dbfs:/mnt/tokyoolympic/raw-data/medals.csv")
medal_df.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
medal_df = medal_df.withColumnRenamed("Rank by Total", "Rank_by_Total")

In [0]:
teams_df = spark.read.format("csv") \
            .option("header","true") \
            .option("inferSchema","true") \
            .load("dbfs:/mnt/tokyoolympic/raw-data/teams.csv")
teams_df.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [0]:
avg_entries_by_gender = entriesGender_df.withColumn("female_ratio", col("female")/col("total")) \
                        .withColumn("male_ratio", col("male")/col("total"))

avg_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|       female_ratio|         male_ratio|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
athelete_df.repartition(1).write.mode("overwrite") \
        .save("dbfs:/mnt/tokyoolympic/transformed-data/atheletes.csv")

In [0]:
coaches_df.repartition(1).write.mode("overwrite") \
        .save("dbfs:/mnt/tokyoolympic/transformed-data/coaches.csv")

In [0]:
entriesGender_df.repartition(1).write.mode("overwrite") \
        .save("dbfs:/mnt/tokyoolympic/transformed-data/entriesgender.csv")

medal_df.repartition(1).write.mode("overwrite") \
        .save("dbfs:/mnt/tokyoolympic/transformed-data/medals.csv")

teams_df.repartition(1).write.mode("overwrite") \
        .save("dbfs:/mnt/tokyoolympic/transformed-data/teams.csv")

In [0]:
athelete_df.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympic/transformed-data/athletes")
     